<a href="https://colab.research.google.com/github/Dineshkumar-Anbalagan/Audio-Sentiment-Analysis/blob/main/Final_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 install pydub
!pip3 install SpeechRecognition

     |████████████████████████████████| 32.8MB 104kB/s 


### **IMPORTING AUDIO BASED LIBRARIES**

In [6]:
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

### **IMPORTING TEXT ANALYTICS AND DATA MANIPULATION LIBRARIES**

In [7]:
#Importing Data Manipulation Library:
import pandas as pd

#Importing Scientific computing library:
import numpy as np

#Importing Plotting libraries:
import matplotlib.pyplot as plt
import seaborn as sns

#Importing Text Analysis Libraries:
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
!pip install textblob 
from nltk.stem.snowball import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download()
from nltk.sentiment.vader import SentimentIntensityAnalyzer as vader
print('Libraries Imported')

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> gutenberg
      Unzipping corpora/gutenberg.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stop

### **IMPORTING ML AND DL LIBRARIES**

In [9]:
#train test split
from sklearn.model_selection import train_test_split

#Deep Learning Libraries
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [10]:
import warnings
warnings.simplefilter('ignore')

In [11]:
data = pd.read_csv("Tweets.csv")
df = data[['airline_sentiment','text']]
df['text'] = df['text'].str.replace('@VirginAmerica', '')
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '',x))

In [12]:
tokenizer  = Tokenizer(num_words=5000, split = " ")
tokenizer.fit_on_texts(df['text'].values)

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [13]:
model = Sequential()
model.add(Embedding(5000, 256, input_length = X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True, dropout = 0.3, recurrent_dropout=0.2))
model.add(LSTM(256, dropout = 0.3, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [14]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
y = pd.get_dummies(df['airline_sentiment']).values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

batch_size = 32
epochs = 8
model.fit(x_train, y_train, epochs=epochs, batch_size = batch_size, verbose = 2)

Epoch 1/8
366/366 - 194s - loss: 0.6524 - accuracy: 0.7292
Epoch 2/8
366/366 - 190s - loss: 0.4360 - accuracy: 0.8303
Epoch 3/8
366/366 - 191s - loss: 0.3474 - accuracy: 0.8711
Epoch 4/8
366/366 - 191s - loss: 0.2866 - accuracy: 0.8916
Epoch 5/8
366/366 - 191s - loss: 0.2337 - accuracy: 0.9137
Epoch 6/8
366/366 - 191s - loss: 0.1948 - accuracy: 0.9285
Epoch 7/8
366/366 - 191s - loss: 0.1640 - accuracy: 0.9380
Epoch 8/8
366/366 - 190s - loss: 0.1422 - accuracy: 0.9483


## **OVERALL INTEGRATED FUNCTION**

In [16]:
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator).lower()

#function to remove stopwords
sw = stopwords.words('english')
def stopwords(text):
    text = [word for word in text.split() if word not in sw]
    return " ".join(text)

stemmer = SnowballStemmer("english")
#a function to stem words
def stemming(text):    
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [17]:
def classify(audio):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    r = sr.Recognizer()

    # open the audio file using pydub
    sound = AudioSegment.from_wav(audio)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except:
                text = f"{text.capitalize()}. "
                #print(chunk_filename, ":", text)
                whole_text += text

    x = remove_punctuation(whole_text) #punctuation removal
    data = stopwords(x) #removing stopwords
    x1 = stemming(data)

    X2 = tokenizer.texts_to_sequences(x1)
    X2 = pad_sequences(X2)

    predictions1 = model.predict(X2)

    pos_count, neu_count, neg_count = 0, 0, 0
    real_pos, real_neu, real_neg = 0, 0, 0
    for i, prediction in enumerate(predictions1):
      if np.argmax(prediction) == 2:
        pos_count += 1
      elif np.argmax(prediction) == 1:
        neu_count += 1
      else:
        neg_count += 1
    return {'Positive': pos_count, 'Neutral': neu_count, 'Negative': neg_count}

In [25]:
classify("test1.wav")

{'Negative': 1, 'Neutral': 4, 'Positive': 1}